In [2]:
import numpy as np
from numba import njit

In [14]:
from pyspi.utils.function_utils import find_response_version
find_response_version("010101 000000")

0

In [3]:
class A:
    def __init__(self):
        self.a = 1
        self.b()
        
    def b(self):
        self.b = (np.array([1,2,3]), np.array([3,4,5]))
        test(self.b)


In [4]:
@njit
def test(a):
    for i in a:
        for j in i:
            print(j)

In [5]:
A()

1
2
3
3
4
5
